In [1]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

ticket_lzj7207


In [2]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

data_sanky.csv	guangdong_station_loc.csv  judge.csv  test.html


In [3]:
# 显示cell运行时长
%load_ext klab-autotime

In [4]:
import pandas as pd
import os,psutil 
import numpy as np
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql import functions

sc = SparkContext()
spark = SparkSession(sc)

time: 3.66 s


In [5]:
df = spark.read.csv('/home/kesci/input/ticket_lzj7207/ticket.csv')
df.show(10) #显示前十行数据

+----+--------+------+----+------------------+----------+----+----+
| _c0|     _c1|   _c2| _c3|               _c4|       _c5| _c6| _c7|
+----+--------+------+----+------------------+----------+----+----+
|null|    null|  null|null|              null|      null|null|null|
|null|    null|  null|   9|              null|      null|null|   9|
|null|    null|  null|   9|              null|      null|null|   9|
|8009|20200222|110000|   1|140225199002140040|大同汽车站|浑源|   1|
|8009|20200222|110000|   1|140225199002140040|大同汽车站|浑源|   1|
|8006|20200224| 92000|   1|140225198309220016|大同汽车站|浑源|   1|
|8006|20200224| 92000|   1|140225198309220016|大同汽车站|浑源|   1|
|8010|20200225|104000|   1|140225199206064318|大同汽车站|浑源|   1|
|8010|20200225|104000|   1|140225199306034319|大同汽车站|浑源|   1|
|8010|20200225|104000|   1|140225199206064318|大同汽车站|浑源|   1|
+----+--------+------+----+------------------+----------+----+----+
only showing top 10 rows

time: 5 s


In [6]:
#查看每一列缺失值的比例
df.agg(*[(1-(F.count(c) /F.count('*'))).alias(c+'_missing') for c in df.columns]).show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|         _c0_missing|         _c1_missing|         _c2_missing|         _c3_missing|        _c4_missing|         _c5_missing|         _c6_missing|         _c7_missing|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+
|1.296325654487340...|3.543911861081383...|3.646498783371754...|8.011106022887127E-6|0.01822224455093946|9.540583773448574E-6|9.382040348171472E-6|2.512446987856798E-5|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+

time: 1min 23s


In [6]:
#去除完全重复的数据
#df = df.drop_duplicates() #没有重复的数据
#筛选列
df = df.drop('_c3').drop('_c4')
#对列重命名
df = df.select(col("_c0").alias("ScheduleCode"), col("_c1").alias("DepartDate"), 
               col("_c2").alias("DepartTime"),col("_c5").alias("StartStationName"),
               col("_c6").alias("ReachStationName"),col("_c7").alias("SeatType"))

#删除缺失值
df = df.dropna()
#展示前十行数据
df.show(10) 

+------------+----------+----------+----------------+----------------+--------+
|ScheduleCode|DepartDate|DepartTime|StartStationName|ReachStationName|SeatType|
+------------+----------+----------+----------------+----------------+--------+
|        8009|  20200222|    110000|      大同汽车站|            浑源|       1|
|        8009|  20200222|    110000|      大同汽车站|            浑源|       1|
|        8006|  20200224|     92000|      大同汽车站|            浑源|       1|
|        8006|  20200224|     92000|      大同汽车站|            浑源|       1|
|        8010|  20200225|    104000|      大同汽车站|            浑源|       1|
|        8010|  20200225|    104000|      大同汽车站|            浑源|       1|
|        8010|  20200225|    104000|      大同汽车站|            浑源|       1|
|        8010|  20200225|    104000|      大同汽车站|            浑源|       1|
|        8006|  20200226|     92000|      大同汽车站|            浑源|       1|
|        8010|  20200226|    104000|      大同汽车站|            浑源|       1|
+------------+----------+-----

In [8]:
df.printSchema()

root
 |-- ScheduleCode: string (nullable = true)
 |-- DepartDate: string (nullable = true)
 |-- DepartTime: string (nullable = true)
 |-- StartStationName: string (nullable = true)
 |-- ReachStationName: string (nullable = true)
 |-- SeatType: string (nullable = true)

time: 3.93 ms


## 座位类型

In [11]:
seat = df.select('SeatType')
seat_group = seat.groupby('SeatType').count() 
seat_group.show(10) 
seat_group.count()

+----------------+-----+
|        SeatType|count|
+----------------+-----+
|  20200227083812|    2|
|  20200227090722|    2|
|  20200228073949|    2|
|20200112095101.0|    2|
|20200123101921.0|    1|
|20200113102922.0|    1|
|20200124185614.0|    1|
|20200120233124.0|    1|
|20200113214016.0|    1|
|20200123112219.0|    1|
+----------------+-----+
only showing top 10 rows



32370

time: 2min 43s


In [12]:
seat_group.toPandas().to_csv("seat_group.csv")

time: 1min 19s


In [13]:
import pandas as pd

time: 1.06 ms


In [24]:
seat_group=pd.read_csv("seat_group.csv")
seat_group2=seat_group.sort_index(axis=0,by="count",ascending=False)
seat_group2.head(15)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  


,Unnamed: 0,SeatType,count
23367,23367,1,65245303
149,149,1.0,26587711
22550,22550,9,12058589
8811,8811,9.0,2788552
29141,29141,3.0,158116
17541,17541,4.0,142405
30600,30600,2,135272
2952,2952,3,21806
26900,26900,4,20776
22879,22879,2.0,14806


time: 52.6 ms


In [31]:
seattype=['1','2','3','4','9']
count=[65245303+26587711+209+176,135272+14806,158116+21806,142405+20776,12058589+2788552]
count

[91833399, 150078, 179922, 163181, 14847141]

time: 4.14 ms


In [32]:
import matplotlib.pyplot as plt
plt.pie(count,labels=seattype)
plt.show()

<Figure size 432x288 with 1 Axes>

time: 70.2 ms


In [34]:
plt.bar(seattype,count)
plt.title("座位类型")
plt.show()

<Figure size 432x288 with 1 Axes>

time: 140 ms


## 乘车站预处理

In [9]:
SSN = df.select('StartStationName') #选择出发车站所对应的列
group = SSN.groupby('StartStationName').count() #将不同车站分组统计
group.show(10) #查看前10个车站及频次
group.count() #一共有4677个出发车站

+------------------------+------+
|        StartStationName| count|
+------------------------+------+
|            太原汽车总站| 10646|
|                  锦州站|221000|
|      白城市中心客运总站| 20257|
|          松原公路客运站| 17082|
|               231123002|     8|
|                万达广场|    91|
|        建州汽车城公交站|     1|
|                    黄石| 37072|
|              芷江汽车站| 10660|
|北海市西区汽车客运服务站|  9184|
+------------------------+------+
only showing top 10 rows



4677

time: 2min 38s


In [10]:
region=pd.read_csv('/home/kesci/input/ticket_lzj7207/行政区划.csv')  #读取行政区划表（另附）
judge = group.select('StartStationName').dropna().toPandas() #将不重复的4677个车站另存为pandas数据框
judge['county']= None
judge['city'] = None
judge['province'] = None

time: 1min 18s


In [11]:
## 有的车站名含有县名，有的只有地级市名，所以共匹配两次。
## 首先精确到county
for i in range(len(judge)):
    for j in range(len(region)):
        if region['county_short'].iloc[j] in judge['StartStationName'].iloc[i]:
            judge['county'][i] = region['county_short'].iloc[j]
            judge['city'][i] = region['city_short'].iloc[j]
            judge['province'][i] = region['province_short'].iloc[j]
            continue
## 没有county的精确到city
for i in range(len(judge)):
    for j in range(len(region)):
        if judge['county'].iloc[i] == None:
            if region['city_short'].iloc[j] in judge['StartStationName'].iloc[i]:
                judge['city'][i] = region['city_short'].iloc[j]
                judge['province'][i] = region['province_short'].iloc[j]
                continue
## 保存judge文件,judge文件包含了4796个车站及其对应的县、地级市、省。
judge.to_csv('/home/kesci/work/judge.csv',encoding='utf_8_sig')

time: 10min 25s


In [12]:
judge.describe()#3794个车站可以查到省份和城市

,StartStationName,county,city,province
count,4677,3040,3794,3794
unique,4677,1601,325,30
top,231225080,文昌,儋州,广东
freq,1,41,250,428


time: 31.5 ms


In [13]:
#匹配出发站对应的省市县
judge = spark.read.csv('/home/kesci/work/judge.csv',header=True, inferSchema=True)
judge=judge.drop('_c0')
df=df.join(judge,'StartStationName') 
df.show()

+----------------+------------+----------+----------+----------------+--------+------+----+--------+
|StartStationName|ScheduleCode|DepartDate|DepartTime|ReachStationName|SeatType|county|city|province|
+----------------+------------+----------+----------+----------------+--------+------+----+--------+
|      大同汽车站|        8009|  20200222|    110000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8009|  20200222|    110000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8006|  20200224|     92000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8006|  20200224|     92000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|  大同|大庆|    黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|  大同|大庆| 

In [14]:
df=df.withColumnRenamed("county","StartCounty").withColumnRenamed("city","StartCity").withColumnRenamed("province","StartProvince")
df.show()

+----------------+------------+----------+----------+----------------+--------+-----------+---------+-------------+
|StartStationName|ScheduleCode|DepartDate|DepartTime|ReachStationName|SeatType|StratCounty|StratCity|StratProvince|
+----------------+------------+----------+----------+----------------+--------+-----------+---------+-------------+
|      大同汽车站|        8009|  20200222|    110000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8009|  20200222|    110000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8006|  20200224|     92000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8006|  20200224|     92000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8010|  20200225|    104000|            浑源|       1|       大同|     大庆|         黑龙|
|      大同汽车站|        8010|  202002

In [17]:
df1=df.filter(df["StartProvince"]=='广东')
df1.show()

+----------------+------------+----------+----------+----------------+--------+-----------+---------+-------------+
|StartStationName|ScheduleCode|DepartDate|DepartTime|ReachStationName|SeatType|StartCounty|StartCity|StartProvince|
+----------------+------------+----------+----------+----------------+--------+-----------+---------+-------------+
|        阜阳西站|      MC0001|  20200413|         N|            董集|       1|       阳西|     阳江|         广东|
|        阜阳西站|      FN0001|  20200413|         N|            阜南|       1|       阳西|     阳江|         广东|
|        阜阳西站|      LQ0001|  20200413|         N|            临泉|       1|       阳西|     阳江|         广东|
|        阜阳西站|      FN0001|  20200413|         N|            阜南|       1|       阳西|     阳江|         广东|
|        阜阳西站|      LQ0001|  20200413|         N|            临泉|       1|       阳西|     阳江|         广东|
|        阜阳西站|      LQ0001|  20200413|         N|            临泉|       1|       阳西|     阳江|         广东|
|        阜阳西站|      FN0001| 

In [18]:
RSN = df1.select('ReachStationName') #选择到达车站所对应的列
group_ = RSN.groupby('ReachStationName').count() #将不同车站分组统计
group_.show(10) #查看前10个车站及频次
group_.count() 

+----------------+-----+
|ReachStationName|count|
+----------------+-----+
|            阜阳| 3038|
|            黄石|  125|
|            洛宁|  559|
|          棉花庄|    3|
|            东平|    1|
|            风台|    1|
|            官桥|  483|
|          界头铺|    4|
|        广州花都| 2327|
|            坪山|  446|
+----------------+-----+
only showing top 10 rows



5987

time: 2min 37s


In [19]:
judge_ = group_.select('ReachStationName').dropna().toPandas() 
judge_['county']= None
judge_['city'] = None
judge_['province'] = None

time: 1min 18s


In [20]:
for i in range(len(judge_)):
    for j in range(len(region)):
        if region['county_short'].iloc[j] in judge_['ReachStationName'].iloc[i]:
            judge_['county'][i] = region['county_short'].iloc[j]
            judge_['city'][i] = region['city_short'].iloc[j]
            judge_['province'][i] = region['province_short'].iloc[j]
            continue

for i in range(len(judge_)):
    for j in range(len(region)):
        if judge_['county'].iloc[i] == None:
            if region['city_short'].iloc[j] in judge_['ReachStationName'].iloc[i]:
                judge_['city'][i] = region['city_short'].iloc[j]
                judge_['province'][i] = region['province_short'].iloc[j]
                continue
judge_.to_csv('/home/kesci/input/ticket_lzj7207/judge_.csv',encoding='utf_8_sig')

time: 14min 21s


In [21]:
judge_.describe()

,ReachStationName,county,city,province
count,5987,3012,3958,3958
unique,5987,990,268,31
top,深圳沙岭,东莞,广州,广东
freq,1,106,268,1854


time: 38.5 ms


In [22]:
#匹配到达站对应的省市县
judge_= spark.read.csv('/home/kesci/input/ticket_lzj7207/judge_.csv',header=True, inferSchema=True)
judge_=judge_.drop('_c0')
df1=df1.join(judge_,'ReachStationName') 
df1.show()

+----------------+----------------+------------+----------+----------+--------+-----------+---------+-------------+------+----+--------+
|ReachStationName|StartStationName|ScheduleCode|DepartDate|DepartTime|SeatType|StartCounty|StartCity|StartProvince|county|city|province|
+----------------+----------------+------------+----------+----------+--------+-----------+---------+-------------+------+----+--------+
|            董集|        阜阳西站|      MC0001|  20200413|         N|       1|       阳西|     阳江|         广东|  null|null|    null|
|            阜南|        阜阳西站|      FN0001|  20200413|         N|       1|       阳西|     阳江|         广东|  阜南|阜阳|    安徽|
|            临泉|        阜阳西站|      LQ0001|  20200413|         N|       1|       阳西|     阳江|         广东|  临泉|阜阳|    安徽|
|            阜南|        阜阳西站|      FN0001|  20200413|         N|       1|       阳西|     阳江|         广东|  阜南|阜阳|    安徽|
|            临泉|        阜阳西站|      LQ0001|  20200413|         N|       1|       阳西|     阳江|         广东|  临泉

In [23]:
df1=df1.withColumnRenamed("county","ReachCounty").withColumnRenamed("city","ReachCity").withColumnRenamed("province","ReachProvince")
df1=df1.dropna()
df1.show()
df1.count()

+----------------+----------------+------------+----------+----------+--------+-----------+---------+-------------+-----------+---------+-------------+
|ReachStationName|StartStationName|ScheduleCode|DepartDate|DepartTime|SeatType|StartCounty|StartCity|StartProvince|ReachCounty|ReachCity|ReachProvince|
+----------------+----------------+------------+----------+----------+--------+-----------+---------+-------------+-----------+---------+-------------+
|            阜南|        阜阳西站|      FN0001|  20200413|         N|       1|       阳西|     阳江|         广东|       阜南|     阜阳|         安徽|
|            临泉|        阜阳西站|      LQ0001|  20200413|         N|       1|       阳西|     阳江|         广东|       临泉|     阜阳|         安徽|
|            阜南|        阜阳西站|      FN0001|  20200413|         N|       1|       阳西|     阳江|         广东|       阜南|     阜阳|         安徽|
|            临泉|        阜阳西站|      LQ0001|  20200413|         N|       1|       阳西|     阳江|         广东|       临泉|     阜阳|         安徽|
|       

1931385

time: 1min 25s


In [24]:
df_guangdong=df1.filter(df1["ReachProvince"]=='广东')
df_guangdong.count()
df_guangdong.toPandas().to_csv('/home/kesci/work/guangdong_station_loc.csv',encoding='utf_8_sig')

time: 3min 2s


In [ ]:
#统计各车站数量
#data_sanky.head()

In [ ]:
#nodes=[]